## 今天的作業
在鐵達尼資料集中，以Titanic_train.csv 中，首先將有遺失值的數值刪除，試著將課堂中所學的方法應用上去。
* Q1: 目標變數為 Survived，以 Pclass, Age, SibSp, Parch, Fare,Sex, Embarked 為特徵，試著用今天教授的包裝法，搭配課程所教的 SVC，試著排出其餘特徵的重要性。


In [1]:
# library
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
import math
import statistics
import seaborn as sns
from IPython.display import display
import sklearn
print(sklearn.__version__)
#如果只有 0.19 記得要更新至 最新版本
print(pd.__version__)
%matplotlib inline




# 特徵選取會用到的函數
from sklearn.feature_selection import VarianceThreshold
from sklearn import preprocessing

from sklearn.datasets import make_friedman1
from sklearn.feature_selection import RFE
from sklearn.svm import SVC

 

0.23.2
1.1.3


## 讀入資料

In [2]:
df_train = pd.read_csv("Titanic_train.csv")
print(df_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None


In [3]:
#1.產稱一個新的變數 Survived_cate ，資料型態傳換成類別型態      
#2.把題目中的 Survived 用 Survived_cate 來做分析    
df_train['Survived_cate']=df_train['Survived']
df_train['Survived_cate']=df_train['Survived_cate'].astype('object')
print(df_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   PassengerId    891 non-null    int64  
 1   Survived       891 non-null    int64  
 2   Pclass         891 non-null    int64  
 3   Name           891 non-null    object 
 4   Sex            891 non-null    object 
 5   Age            714 non-null    float64
 6   SibSp          891 non-null    int64  
 7   Parch          891 non-null    int64  
 8   Ticket         891 non-null    object 
 9   Fare           891 non-null    float64
 10  Cabin          204 non-null    object 
 11  Embarked       889 non-null    object 
 12  Survived_cate  891 non-null    object 
dtypes: float64(2), int64(5), object(6)
memory usage: 90.6+ KB
None


In [4]:
#我們先把遺失值刪除
## 取出資料後，把遺失值刪除
complete_data=df_train.dropna()

#排除 Ｎame 當特徵，先刪除每一個人都是獨特的資料，先檢視其他變數
complete_data=complete_data.drop(['Name','Ticket','PassengerId'], axis=1)
display(complete_data.head(5))

print(complete_data.shape)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Survived_cate
1,1,1,female,38.0,1,0,71.2833,C85,C,1
3,1,1,female,35.0,1,0,53.1000,C123,S,1
6,0,1,male,54.0,0,0,51.8625,E46,S,0
10,1,3,female,4.0,1,1,16.7000,G6,S,1
11,1,1,female,58.0,0,0,26.5500,C103,S,1


(183, 10)


In [5]:
num_features = []
for dtype, feature in zip(complete_data.dtypes, complete_data.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

6 Numeric Features : ['Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare']



In [6]:
cat_features = []
for dtype, feature in zip(complete_data.dtypes, complete_data.columns):
    if dtype == 'object':
        cat_features.append(feature)
print(f'{len(cat_features)} category Features : {cat_features}\n')

4 category Features : ['Sex', 'Cabin', 'Embarked', 'Survived_cate']



## 定義 x,y

In [7]:
x=complete_data[['Pclass', 'Age', 'SibSp', 'Parch', 'Fare','Sex', 'Embarked']]
y=complete_data['Survived']
display(x.head(5))
display(y.head(5))

,Pclass,Age,SibSp,Parch,Fare,Sex,Embarked
1,1,38.0,1,0,71.2833,female,C
3,1,35.0,1,0,53.1000,female,S
6,1,54.0,0,0,51.8625,male,S
10,3,4.0,1,1,16.7000,female,S
11,1,58.0,0,0,26.5500,female,S


1     1
3     1
6     0
10    1
11    1
Name: Survived, dtype: int64

### Q1: 目標變數為 Survived，試著用今天教授的包裝法，搭配課程所教的 SVC，試著排出其餘特徵的重要性!


In [23]:
#離散要轉換成數值，['Sex', 'Embarked']
# Sex
sex_mapping = {
           'male': 1,
           'female': 0}
complete_data['Sex1'] = complete_data['Sex'].map(sex_mapping).astype('object')

# embarked
embarked_mapping = {
           'C': 1,
           'S': 0}
complete_data['Embarked1'] = complete_data['Embarked'].map(embarked_mapping).astype('object')

# Pclass, Parch
complete_data['Pclass_cate']=complete_data['Pclass'].astype('object')
complete_data['Parch_cate']=complete_data['Parch'].astype('object')
complete_data['SibSp_cate']=complete_data['SibSp'].astype('object')

# Sex+Age
def age_map2(row): 
    if (row.Sex=='female'):
        return('1')
    else: 
        if(row.Age>=65):
            return('1')
        else:
            return('0')

# Fare
display(complete_data.Fare.describe())
complete_data['Age_above65_female']=complete_data[['Age','Sex']].apply(age_map2,axis=1)

def fare_map(x): 
    if x <= 30:
        return('0')
    elif x <= 60: 
        return('1')
    elif x <= 90: 
        return('2')
    else:
        return('3')

complete_data['Fare_cate']=complete_data['Fare'].apply(fare_map)

display(complete_data)
print(complete_data.info())





count    183.000000
mean      78.682469
std       76.347843
min        0.000000
25%       29.700000
50%       57.000000
75%       90.000000
max      512.329200
Name: Fare, dtype: float64

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Survived_cate,Sex1,Embarked1,Pclass_cate,Parch_cate,Age_above65_female,Fare_cat,SibSp_cate,Fare_cate
1,1,1,female,38.0,1,0,71.2833,C85,C,1,0,1,1,0,1,2,1,2
3,1,1,female,35.0,1,0,53.1000,C123,S,1,0,0,1,0,1,1,1,1
6,0,1,male,54.0,0,0,51.8625,E46,S,0,1,0,1,0,0,1,0,1
10,1,3,female,4.0,1,1,16.7000,G6,S,1,0,0,3,1,1,0,1,0
11,1,1,female,58.0,0,0,26.5500,C103,S,1,0,0,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
871,1,1,female,47.0,1,1,52.5542,D35,S,1,0,0,1,1,1,1,1,1
872,0,1,male,33.0,0,0,5.0000,B51 B53 B55,S,0,1,0,1,0,0,0,0,0
879,1,1,female,56.0,0,1,83.1583,C50,C,1,0,1,1,1,1,2,0,2
887,1,1,female,19.0,0,0,30.0000,B42,S,1,0,0,1,0,1,0,0,0


<class 'pandas.core.frame.DataFrame'>
Int64Index: 183 entries, 1 to 889
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Survived            183 non-null    int64  
 1   Pclass              183 non-null    int64  
 2   Sex                 183 non-null    object 
 3   Age                 183 non-null    float64
 4   SibSp               183 non-null    int64  
 5   Parch               183 non-null    int64  
 6   Fare                183 non-null    float64
 7   Cabin               183 non-null    object 
 8   Embarked            183 non-null    object 
 9   Survived_cate       183 non-null    object 
 10  Sex1                183 non-null    object 
 11  Embarked1           181 non-null    object 
 12  Pclass_cate         183 non-null    object 
 13  Parch_cate          183 non-null    object 
 14  Age_above65_female  183 non-null    object 
 15  Fare_cat            183 non-null    object 
 16  SibSp_ca

In [27]:

#顯示所有行
pd.set_option('display.max_columns', None)
#顯示所有列
pd.set_option('display.max_rows', None)
#顯示行設定
pd.set_option('max_colwidth',100)

complete_data=complete_data.dropna()

#執行 包裝法
from sklearn.datasets import make_friedman1
from sklearn.feature_selection import RFE
from sklearn.svm import SVC
#ValueError: could not convert string to float: 'Male'

x=complete_data[['Pclass_cate', 'Age_above65_female', 'SibSp_cate', 'Parch_cate', 'Fare_cate','Sex1', 'Embarked1']]
display(x)

y=complete_data['Survived']

estimator = SVC(kernel="linear")
selector = RFE(estimator, n_features_to_select=3, step=1)
selector = selector.fit(x, y)
## True= selected feature
print(selector.support_)
#Selected (i.e., estimated best) features are assigned rank 1.
ranking=selector.ranking_
print(ranking)
rfe_feature = x.loc[:,selector.support_].columns.tolist()
print(rfe_feature)


,Pclass_cate,Age_above65_female,SibSp_cate,Parch_cate,Fare_cate,Sex1,Embarked1
1,1,1,1,0,2,0,1
3,1,1,1,0,1,0,0
6,1,0,0,0,1,1,0
10,3,1,1,1,0,0,0
11,1,1,0,0,0,0,0
21,2,0,0,0,0,1,0
23,1,0,0,0,1,1,0
27,1,0,3,2,3,1,0
52,1,1,1,0,2,0,1
54,1,1,0,1,2,1,1


[False  True False  True False  True False]
[2 1 4 1 3 1 5]
['Age_above65_female', 'Parch_cate', 'Sex1']
